# Data storage and project layout

- The initial dataset should be stored as `mol2` or `mol2.gz` files in a directory. For example:

In [8]:
!ls -lh .

total 2343768
-rw-r--r--  1 sebastian  staff   572M May  6 21:20 1.mol2
-rw-r--r--  1 sebastian  staff   572M May  6 21:22 2.mol2


- for a new project, it's best to create a new directory, for instance, we create a new subdirectory in 80698:

In [2]:
!mkdir 80698/proj-1/

Using the `count_mol2.py` script, we can count the number of structures in each mol2 file in the input directory:

In [9]:
!python ../../../tools/count_mol2.py -i 80698/mol2

1.mol2 : 135966
2.mol2 : 136144
Total : 272110


Optionally, you may have files with data about the molecules, for instance:

In [ ]:
!ls -lh 80698/csv/

# General Blacklist & Whitelist filtering

- `mol2_to_id.py` generates a list of molecule IDs from MOL2 files
- `id_to_mol2.py` filters mol2 files by IDs and creates new mol2 files. Via whitelisting, molecules that match those IDs are written. Via blacklisting, all molecules but the molecules that are in the list are written

In [11]:
!python ../../../tools/mol2_to_id.py -i 80698/mol2 -o 80698/proj-1/all-mol2ids.txt

Processing 80698/mol2/1.mol2
Processing 80698/mol2/2.mol2
Finished


In [12]:
!head 80698/proj-1/all-mol2ids.txt

ZINC57271411
ZINC50764925
ZINC65255333
ZINC06394508
ZINC65292537
ZINC65375610
ZINC31820077
ZINC65395084
ZINC00205726
ZINC01458151


- Say we are interested in a subset of molecules only. Consider this example: 1st we create a list of IDs:

In [7]:
%%writefile ./80698/proj-1/selected-example-mol2ids.txt
ZINC57271411
ZINC06394508
ZINC65292537
ZINC31820077
ZINC01458151

Overwriting ./80698/proj-1/selected-example-mol2ids.txt


### Whitelisting

In [23]:
!python ../../../tools/id_to_mol2.py \
--input 80698/mol2 \
--output 80698/proj-1/selected-example-mol2ids \
--id_file ./80698/proj-1/selected-example-mol2ids.txt \
--whitelist True

Processing 80698/mol2/1.mol2
Processing 80698/mol2/2.mol2
Finished


Now, the output directory, `80698/proj-1/selected-example-mol2ids`, should contain only mol2 files with the selected IDs:

In [24]:
!ls 80698/proj-1/selected-example-mol2ids

1.mol2 2.mol2


In [25]:
!grep 'ZINC' 80698/proj-1/selected-example-mol2ids/1.mol2

ZINC57271411
ZINC06394508
ZINC65292537
ZINC31820077
ZINC01458151


In [28]:
!grep -C 1 'ZINC' 80698/proj-1/selected-example-mol2ids/1.mol2 

@<TRIPOS>MOLECULE
ZINC57271411
   50    52     0     0     0
--
--
@<TRIPOS>MOLECULE
ZINC06394508
   49    52     0     0     0
--
--
@<TRIPOS>MOLECULE
ZINC65292537
   47    50     0     0     0
--
--
@<TRIPOS>MOLECULE
ZINC31820077
   48    50     0     0     0
--
--
@<TRIPOS>MOLECULE
ZINC01458151
   25    25     0     0     0


### Blacklisting

Similar to the previous approach, using a whitelist filter, we can do blacklist filtering, which means that all molecules are selected but the ones contained in the ID files. Set whitelist to False.

In [46]:
!python ../../../tools/id_to_mol2.py \
--input 80698/mol2 \
--output 80698/proj-1/nonselected-example-mol2ids \
--id_file ./80698/proj-1/selected-example-mol2ids.txt \
--whitelist false

Processing 80698/mol2/1.mol2
Processing 80698/mol2/2.mol2
Finished


In [47]:
!python ../../../tools/count_mol2.py -i 80698/proj-1/nonselected-example-mol2ids/

1.mol2 : 135961
2.mol2 : 136144
Total : 272105


# First Filtering Step -- Filtering via Features from Data Tables

In [2]:
import time

In [48]:
!head 80698/csv/3_prop.xls

ZINC_ID	MWT	LogP	Desolv_apolar	Desolv_polar	HBD	HBA	tPSA	Charge	NRB	SMILES
ZINC00000010	217.2	1.42	5.57	-41.98	0	4	66	-1	2	C[C@@]1(C(=O)C=C(O1)C(=O)[O-])c2ccccc2
ZINC00000012	289.356	1.28	4.89	-24.55	2	4	66	0	5	c1ccc(cc1)C(c2ccccc2)[S@](=O)CC(=O)NO
ZINC00000017	281.337	1.33	3.06	-23.33	2	6	87	0	4	CCC[S@](=O)c1ccc2c(c1)[nH]/c(=N\C(=O)OC)/[nH]2
ZINC00000017	281.337	1.33	3.07	-19.2	2	6	87	0	4	CCC[S@](=O)c1ccc2c(c1)[nH]/c(=N/C(=O)OC)/[nH]2
ZINC00000018	212.318	2.00	5.87	-8.2	1	3	32	0	4	CC(C)C[C@@H]1C(=O)N(C(=S)N1)CC=C
ZINC00000021	288.411	3.85	4.02	-40.52	1	3	30	1	6	CCC(=O)O[C@]1(CC[NH+](C[C@@H]1CC=C)C)c2ccccc2
ZINC00000022	218.276	3.21	0.47	-48.57	1	3	52	-1	5	C[C@@H](c1ccc(cc1)NCC(=C)C)C(=O)[O-]
ZINC00000025	251.353	3.60	2.4	-41.56	2	2	40	1	5	C[C@H](Cc1ccccc1)[NH2+][C@@H](C#N)c2ccccc2
ZINC00000030	297.422	2.94	0.89	-37.97	3	3	47	1	6	C[C@@H](CC(c1ccccc1)(c2ccccc2)C(=O)N)[NH+](C)C


- A valid query looks like this:
 
- Correct: `"(MWT >= 200) & (NRB <= 7)"`
- Wrong: `"( MWT >= 200) & ( NRB <= 7)"` [spacing between parentheses and column names]
- Wrong: `"MWT >= 200 & NRB <= 7"` [expressions seperated by logical '&' operator not enclosed in parentheses]
- Wrong: `"(mwt >= 200) & (nrb <= 7)"` [column names don't match the columns in the data table file]
- Wrong: `"(mwt>=200) & (nrb<=7)"` [no whitespace before and after operators for comparison]


In [107]:
!python ../../../tools/datatable_to_id.py \
--input 80698/csv/3_prop.xls \
--output 80698/proj-1/selected-mol2ids.txt \
--id_column ZINC_ID \
--selection "(NRB <= 7) & (MWT >= 200)"

Using columns: ['ZINC_ID', 'NRB', 'MWT']
Using selection: (chunk.NRB <= 7) & (chunk.MWT >= 200)
Processed: 18000000
Selected: 17599186


Next, we are going to use these IDs to select molecules from the existing mol2 files:

In [108]:
!python ../../../tools/id_to_mol2.py \
--input 80698/mol2 \
--output 80698/proj-1/selected-mol2s \
--id_file ./80698/proj-1/selected-mol2ids.txt \
--whitelist True

Processing 80698/mol2/1.mol2
Processing 80698/mol2/2.mol2
Finished


In [109]:
!python ../../../tools/count_mol2.py -i 80698/proj-1/selected-mol2ids/

1.mol2 : 133655
2.mol2 : 133607
Total : 267262


# Second Filtering Step -- Presence and absence of functional groups

- allowed columns
  - atom_id
  - atom_name
  - atom_type
  - subst_id
  - subst_name
  - charge
  
(may change with updated biopandas)

a) simultaneously true (here impossible): "((atom_type == 'S.3') | (atom_type == 'S.o2')) & (atom_type == 'O.2')"

- tries to find an S.3 or S.o2 atom that is also an O.2 atom at the same time

b) simultaneously true and valid: "((atom_type == 'S.3') | (atom_type == 'S.o2')) & (charge < 0.0)"

- tries to find an S.3 or S.o2 atom that has a negative charge

c) otherwise, "((atom_type == 'S.3') | (atom_type == 'S.o2')) --> (atom_type == 'O.2')"

- tries to find an S.3 or S.o2 atom, then tries to find an O.2 atom as well

d) when in doubt, one can run (as alternative for c) the tool 2 times. 1 time to select "(atom_type == 'S.3') | (atom_type == 'S.o2')" and a second time to select "(atom_type == 'O.2')".

In [4]:
start = time.time()

!python ../../../tools/funcgroup_to_id.py \
--input 80698/proj-1/selected-mol2s/ \
--output 80698/proj-1/3keto-and-sulfur-mol2ids.txt \
--selection "((atom_type == 'S.3') | (atom_type == 'S.o2')) --> (atom_type == 'O.2')" \
--processes 0

time.time() - start

Processing 80698/proj-1/selected-mol2s/1.mol2
Processing 80698/proj-1/selected-mol2s/2.mol2


800.6091911792755

In [5]:
start = time.time()

!python ../../../tools/id_to_mol2.py \
--input 80698/proj-1/selected-mol2s/ \
--output 80698/proj-1/3keto-and-sulfur-mol2s \
--id_file 80698/proj-1/3keto-and-sulfur-mol2ids.txt \
--whitelist True

time.time() - start

Processing 80698/proj-1/selected-mol2s/1.mol2
Processing 80698/proj-1/selected-mol2s/2.mol2
Finished


16.29014277458191

In [7]:
!python ../../../tools/count_mol2.py -i 80698/proj-1/3keto-and-sulfur-mol2s

1.mol2 : 38558
2.mol2 : 38583
Total : 77141


# Third Filtering Step -- Distance between functional groups

- could be done after omega

In [21]:
start = time.time()

!python ../../../tools/funcgroup_distance_to_id.py \
--input 80698/proj-1/3keto-and-sulfur-mol2s/ \
--output 80698/proj-1/3keto-and-sulfur_distance-mol2ids.txt \
--selection "((atom_type == 'S.3') | (atom_type == 'S.o2')) --> (atom_type == 'O.2')" \
--distance "13-20" \
--processes 0

time.time() - start

Processing 80698/proj-1/3keto-and-sulfur-mol2s/1.mol2
Processing 80698/proj-1/3keto-and-sulfur-mol2s/2.mol2


476.1484110355377

In [22]:
start = time.time()

!python ../../../tools/id_to_mol2.py \
--input 80698/proj-1/3keto-and-sulfur-mol2s \
--output 80698/proj-1/3keto-and-sulfur-mol2s_13-20A \
--id_file 80698/proj-1/3keto-and-sulfur_distance-mol2ids.txt \
--whitelist True

time.time() - start

Processing 80698/proj-1/3keto-and-sulfur-mol2s/1.mol2
Processing 80698/proj-1/3keto-and-sulfur-mol2s/2.mol2
Finished


6.941205024719238

In [23]:
!python ../../../tools/count_mol2.py -i 80698/proj-1/3keto-and-sulfur-mol2s_13-20A

1.mol2 : 467
2.mol2 : 461
Total : 928


# Generating Conformers via Omega

In [ ]:
!python ../../../tools/run_omega.py \
--input 80698/proj-1/3keto-and-sulfur-mol2s_13-20A \
--output 80698/proj-1/3keto-and-sulfur-mol2s_13-20A_confs \
--executable "/Applications/OMEGA 2.5.1.4.app/Contents/MacOS/omega2-2.5.1.4" \
--processes 0

Processing 80698/proj-1/3keto-and-sulfur-mol2s_13-20A/1.mol2
Processing 80698/proj-1/3keto-and-sulfur-mol2s_13-20A/2.mol2

          :jGf:             .d8888b. 88d8b.d8b. .d8888b. .d8888b. .d8888b.
        :jGDDDDf:           88'  `88 88'`88'`88 88ooood8 88'  `88 88'  `88
      ,fDDDGjLDDDf,         88.  .88 88  88  88 88.  ... 88.  .88 88.  .88
    ,fDDLt:   :iLDDL;       `88888P' dP  dP  dP `88888P' `8888P88 `88888P8
  ;fDLt:         :tfDG;                                       .88
,jft:   ,ijfffji,   :iff                                  d8888P
     .jGDDDDDDDDDGt.      
    ;GDDGt:''':tDDDG,          Copyright (c) 2004-2013
   .DDDG:       :GDDG.         OpenEye Scientific Software, Inc.
   ;DDDj         tDDDi    
   ,DDDf         fDDD,         Version: 2.5.1.4
    LDDDt.     .fDDDj          Built:   20130515
    .tDDDDfjtjfDDDGt           OEChem version: 1.9.1
      :ifGDDDDDGfi.            Platform: osx-10.8-clang++4-x64
          .:::.                
  ......................   

In [51]:
import subprocess

cmd = ['/Applications/OMEGA\ 2.5.1.4.app/Contents/MacOS/omega2-2.5.1.4', '-in', '80698/proj-1/3keto-and-sulfur-mol2s_13-20A/1.mol2', '-out', '80698/proj-1/3keto-and-sulfur-mol2s_13-20A_confs/1.mol2', '-prefix', '80698/proj-1/3keto-and-sulfur-mol2s_13-20A_confs/1', '-maxconfs', '200', '-warts', 'false']
p = subprocess.Popen(cmd, stderr=subprocess.PIPE, bufsize=1)
for line in p.stderr:
    print(line)

p.stderr.close()
p.wait()


FileNotFoundError: [Errno 2] No such file or directory: '/Applications/OMEGA\\ 2.5.1.4.app/Contents/MacOS/omega2-2.5.1.4'

In [42]:
!/Applications/OMEGA\ 2.5.1.4.app/Contents/MacOS/omega2-2.5.1.4 -in 80698/proj-1/3keto-and-sulfur-mol2s_13-20A/1.mol2 -out 80698/proj-1/3keto-and-sulfur-mol2s_13-20A_confs/1.mol2 -prefix 80698/proj-1/3keto-and-sulfur-mol2s_13-20A_confs/1 -maxconfs 200 -warts false

LICENSE: Could not open license file "oe_license.txt" in local directory
LICENSE: N.B. OE_LICENSE environment variable is not set
LICENSE: N.B. OE_DIR environment variable is not set

          :jGf:             .d8888b. 88d8b.d8b. .d8888b. .d8888b. .d8888b.
        :jGDDDDf:           88'  `88 88'`88'`88 88ooood8 88'  `88 88'  `88
      ,fDDDGjLDDDf,         88.  .88 88  88  88 88.  ... 88.  .88 88.  .88
    ,fDDLt:   :iLDDL;       `88888P' dP  dP  dP `88888P' `8888P88 `88888P8
  ;fDLt:         :tfDG;                                       .88
,jft:   ,ijfffji,   :iff                                  d8888P
     .jGDDDDDDDDDGt.      
    ;GDDGt:''':tDDDG,          Copyright (c) 2004-2013
   .DDDG:       :GDDG.         OpenEye Scientific Software, Inc.
   ;DDDj         tDDDi    
   ,DDDf         fDDD,         Version: 2.5.1.4
    LDDDt.     .fDDDj          Built:   20130515
    .tDDDDfjtjfDDDGt           OEChem version: 1.9.1
      :ifGDDDDDGfi.            Platform: osx-10.8-clang++4-x6

In [40]:
!"$omega2"

/bin/sh: : command not found
